### Make sure grafwiz is installed

In [ ]:
!pip install git+https://github.com/v3io/grafwiz --upgrade

### Add environment configurations and import the necessary libraries
* Please make sure you have a Grafana service, in order to create one, please go to Iguazio dashboard -> Service, Create new service
* To retrieve the url of Grafana service please use "API" tab, in the services dashboard

In [ ]:
import os

grafana_url = 'http://grafana' 
v3io_container = 'users'
stocks_kv_table = os.path.join(os.getenv("V3IO_USERNAME"),'stocks_kv_table')
stocks_tsdb_table = os.path.join(os.getenv("V3IO_USERNAME"),'stocks_tsdb_table')
sym = 'XYZ'
rows = 3450


In [ ]:
from grafwiz import *
import v3io_frames as v3f
import pandas as pd

### Generate data for the dataframe

#### Date that will serve as an index for the time series and additional parameters

In [ ]:
import random
import datetime
def generate_date(rows):
    datetimes = [datetime.datetime.today() - (random.random() * datetime.timedelta(minutes=15)) for i in range(rows)]
    return datetimes

def generate_volume_price(rows):
    volume = [random.randint(0,10000) for i in range(rows)]
    return volume

time = generate_date(rows)
volume = generate_volume_price(rows)
price = generate_volume_price(rows)


#### Create a dataframe out of the generated index and add an index to the dataframe in order to make it accessible for tsdb

In [ ]:
stocks_df = pd.DataFrame(
    {'last_updated': time,
     'volume': volume,
     'price': price
    })
stocks_df['symbol'] = sym
stocks_df = stocks_df.sort_values('last_updated')

stocks_df

In [ ]:
stocks_df_tsdb = stocks_df
stocks_df_tsdb = stocks_df.reset_index()
stocks_df_tsdb = stocks_df.set_index(['last_updated'])


#### Create a client for v3io_frames create table and persist the dataframe in tsdb

In [ ]:
client = v3f.Client('framesd:8081',container=v3io_container)
client.create(backend='tsdb', table=stocks_tsdb_table, rate='1/m', if_exists=1)
client.write(backend='tsdb', table=stocks_tsdb_table, dfs=stocks_df_tsdb)

#### Update the kv table in order of rows arrival to simulate real time changing data

In [ ]:
expr_template = "symbol='{symbol}';price='{price}';volume='{volume}';last_updated='{last_updated}'"
# update NoSQL table with stock data
for idx, record in stocks_df.iterrows():
    stock = {'symbol': sym, 'price': record['price'], 'volume': record['volume'], 'last_updated': record['last_updated']}
    expr = expr_template.format(**stock)
    client.execute('kv', stocks_kv_table, 'update', args={'key': sym, 'expression': expr})

#### Create a Datasource for grafana table and check that kv table exists and accessible

In [ ]:
# Create datasources
DataSource(name='Iguazio').deploy(grafana_url, use_auth=True)

# Verify the KV table can be shown
client = v3f.Client('framesd:8081', container=v3io_container)
client.execute(backend='kv', table=stocks_kv_table, command='infer')

#### Create a Dashboard define its datasource

In [ ]:
# Create grafana dashboard
dash = Dashboard("stocks", start='now', dataSource='Iguazio')


#### Create a table and 2 graphs
Please note that you have to wait for couple of minutes to see the graphs being updated

In [ ]:
# Create a table and log viewer in one row
tbl = Table('Current Stocks Value', span=12).source(table=stocks_kv_table,fields=['symbol','volume', 'price', 'last_updated'],container=v3io_container)
dash.row([tbl])

# Metrics row will
metrics_row = [Graph(metric).series(table=stocks_tsdb_table, fields=[metric], container=v3io_container) for metric in ['price','volume']]

dash.row(metrics_row)



In [ ]:
# Deploy to Grafana
dash.deploy(grafana_url)